## Basic Image Preprocessing, CNN and Transfer Learning

Get the current working directory

In [3]:
import os

PATH = os.getcwd()

Define the data path

In [4]:
# Define data path
data_path = PATH + '/data'
data_dir_list = os.listdir(data_path)

In [5]:
print(data_dir_list)

['Humans', 'dogs', 'cats', 'horses']


Required variables declaration and initialization

In [6]:
img_rows=224
img_cols=224
num_channel=3
num_epoch=2

img_data_list=[]
classes_names_list=[]

Read the images and store them in the list

In [7]:
import cv2

for dataset in data_dir_list:
    classes_names_list.append(dataset) 
    print ('Loading images from {} folder\n'.format(dataset)) 
    img_list=os.listdir(data_path+'/'+ dataset)
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(img_rows, img_cols))
        img_data_list.append(input_img_resize)

Loading images from Humans folder

Loading images from dogs folder

Loading images from cats folder

Loading images from horses folder



In [8]:
num_classes = len(classes_names_list)

###### Image preprocessiong 

In [9]:
import numpy as np
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255

In [10]:
print (img_data.shape)

img_data = img_data.reshape(img_data.shape[0], img_data.shape[1], img_data.shape[2], num_channel)

(808, 224, 224, 3)


In [11]:
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape

In [12]:
classes = np.ones((num_of_samples,), dtype='int64')
classes[0:202]=0
classes[202:404]=1
classes[404:606]=2
classes[606:]=3

In [13]:
from keras.utils import to_categorical

# convert class labels to on-hot encoding
classes = to_categorical(classes, num_classes)

/home/yogesh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
Use the retry module or similar alternatives.


Shuffle the dataset

In [14]:
from sklearn.utils import shuffle

x, y = shuffle(img_data, classes, random_state=2)

Split the dataset

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

Defining the model

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [17]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


Compile the model

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=["accuracy"])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Viewing model_configuration

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 108, 108, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 106, 106, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 53, 53, 64)        0         
__________

In [20]:
model.layers[1].get_weights()[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [21]:
model.get_config()

[{'class_name': 'Conv2D',
  'config': {'activation': 'relu',
   'activity_regularizer': None,
   'batch_input_shape': (None, 224, 224, 3),
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'data_format': 'channels_last',
   'dilation_rate': (1, 1),
   'dtype': 'float32',
   'filters': 32,
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'distribution': 'uniform',
     'mode': 'fan_avg',
     'scale': 1.0,
     'seed': None}},
   'kernel_regularizer': None,
   'kernel_size': (3, 3),
   'name': 'conv2d_1',
   'padding': 'valid',
   'strides': (1, 1),
   'trainable': True,
   'use_bias': True}},
 {'class_name': 'Conv2D',
  'config': {'activation': 'relu',
   'activity_regularizer': None,
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'data_format': 'channels_last',
   'dilation_rate': (1, 1

In [22]:
model.layers[0].get_config()

{'activation': 'relu',
 'activity_regularizer': None,
 'batch_input_shape': (None, 224, 224, 3),
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'data_format': 'channels_last',
 'dilation_rate': (1, 1),
 'dtype': 'float32',
 'filters': 32,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'kernel_size': (3, 3),
 'name': 'conv2d_1',
 'padding': 'valid',
 'strides': (1, 1),
 'trainable': True,
 'use_bias': True}

In [23]:
model.layers[0].input_shape

(None, 224, 224, 3)

In [24]:
model.layers[0].output_shape

(None, 222, 222, 32)

In [25]:
model.layers[0].get_weights()

[array([[[[ 0.03945214, -0.07892564,  0.01051739,  0.11636846,
            0.02723001,  0.01414673,  0.10321297, -0.13023351,
           -0.08340718,  0.09130009, -0.0866773 , -0.07779272,
           -0.10364044,  0.0288492 , -0.02880049, -0.10375497,
           -0.09007759,  0.10980056,  0.0813418 , -0.12433853,
           -0.05767737,  0.10391589, -0.08611361, -0.10129665,
            0.10460362,  0.07335152,  0.10418056, -0.0008291 ,
           -0.11818038,  0.10983758, -0.07592501, -0.10735293],
          [ 0.11049823,  0.0239237 ,  0.05504836,  0.07134505,
            0.04668747,  0.01604337, -0.10452241, -0.01338176,
            0.01933366, -0.0856971 , -0.00940448,  0.11973087,
           -0.09462555, -0.0944702 , -0.02179252,  0.0875461 ,
           -0.07609825,  0.0518734 , -0.01782826, -0.1139599 ,
            0.05473679,  0.11863779,  0.09188713,  0.0623903 ,
           -0.02952177,  0.11748646, -0.13757706, -0.07899509,
           -0.01067145, -0.13790333,  0.01881206, -0.0

In [26]:
print(np.shape(model.layers[0].get_weights()))
print(np.shape(model.layers[0].get_weights()[0]))
print(np.shape(model.layers[0].get_weights()[1]))

(2,)
(3, 3, 3, 32)
(32,)


In [27]:
model.layers[0].trainable

True

#### Training/fit the model 

In [28]:
hist = model.fit(X_train, y_train, batch_size=16, epochs=num_epoch, verbose=1, validation_data=(X_test, y_test))

Train on 646 samples, validate on 162 samples
Epoch 1/2
646/646 [==============================] - 100s - loss: 1.6481 - acc: 0.2539 - val_loss: 1.4039 - val_acc: 0.2469
Epoch 2/2
646/646 [==============================] - 97s - loss: 1.3917 - acc: 0.2910 - val_loss: 1.2995 - val_acc: 0.3765


##### Evaluating the model

In [29]:
score = model.evaluate(X_test, y_test, batch_size=16)
print('Test Loss:', score[0])
print('Test Accuracy:', score[1])

162/162 [==============================] - 7s     
Test Loss: 1.2995230474589783
Test Accuracy: 0.3765432098765432


In [30]:
test_image = X_test[0:1]
print (test_image.shape)

(1, 224, 224, 3)


In [31]:
print(model.predict(test_image))
print(model.predict_classes(test_image))
print(y_test[0:1])

[[0.2331537  0.2536217  0.24537747 0.26784712]]
1/1 [==============================] - 0s
[3]
[[0. 0. 1. 0.]]


##### Printing the confusion matrix

In [32]:
from sklearn.metrics import confusion_matrix

Y_pred = model.predict(X_test)
print(Y_pred)

[[0.2331537  0.2536217  0.24537745 0.26784712]
 [0.20532821 0.2548991  0.23188284 0.3078899 ]
 [0.22795032 0.2513079  0.2402279  0.28051382]
 [0.15763423 0.22665913 0.21252245 0.40318418]
 [0.05415497 0.16666207 0.11741827 0.6617647 ]
 [0.20328061 0.2502302  0.23193586 0.31455338]
 [0.22768117 0.25040764 0.24503109 0.27688015]
 [0.19602336 0.24534641 0.22525537 0.3333749 ]
 [0.25094998 0.25078276 0.2515188  0.24674846]
 [0.0579782  0.16873285 0.10451636 0.6687725 ]
 [0.24234584 0.25273886 0.2506263  0.25428897]
 [0.15073735 0.24696931 0.19714963 0.40514365]
 [0.24833934 0.25056022 0.25175077 0.2493497 ]
 [0.15701985 0.23554577 0.19939213 0.4080422 ]
 [0.25233942 0.25004372 0.2563575  0.24125938]
 [0.24568534 0.25196114 0.25089428 0.25145927]
 [0.19113874 0.2616466  0.22243468 0.32478002]
 [0.1843046  0.24541928 0.21612705 0.35414907]
 [0.25054348 0.24919462 0.2539646  0.24629724]
 [0.2525434  0.24994789 0.2553108  0.24219798]
 [0.15877096 0.23449908 0.20227596 0.404454  ]
 [0.23528734 

In [33]:
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)

[3 3 3 3 3 3 3 3 2 3 3 3 2 3 2 1 3 3 2 2 3 3 3 3 2 3 3 3 1 3 2 3 3 2 3 3 2
 2 3 3 3 3 3 3 2 3 2 3 3 3 2 3 3 2 3 3 0 2 3 3 3 2 3 3 3 3 3 2 3 3 3 3 2 3
 3 3 3 3 1 3 2 3 3 2 3 3 3 3 3 2 3 3 2 3 2 2 3 2 3 3 3 3 2 3 2 3 1 3 3 2 3
 3 3 3 3 3 3 2 3 3 2 2 2 3 2 3 3 3 1 3 2 2 3 3 3 0 3 2 3 3 3 3 3 3 3 2 3 3
 3 3 3 3 0 3 3 3 3 3 3 1 3 3]


In [34]:
print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

[[ 3  3  7 28]
 [ 0  1  9 30]
 [ 0  2 15 16]
 [ 0  0  6 42]]


###### Saving and loading model and weights

In [35]:
from keras.models import model_from_json, load_model

In [36]:
# serialize model to JSON
model_json = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model.h5")

In [37]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")

In [38]:
model.save('model.hdf5')
loaded_model=load_model('model.hdf5')

###### Transfer Learning - 1

In [39]:
from keras.layers import Input, Dense

In [40]:
# Custom_vgg_model_1
#Training the classifier alone
image_input = Input(shape=(224, 224, 3))

In [41]:
from keras.applications.vgg16 import VGG16

model = VGG16(input_tensor=image_input, include_top=True, weights='imagenet')

553074688/553467096 [============================>.] - ETA: 0s


include_top: whether to include the 3 fully-connected layers at the top of the network.

weights: one of None (random initialization) or 'imagenet' (pre-training on ImageNet).

input_tensor: optional Keras tensor (i.e. output of layers.Input()) to use as image input for the model.

classes: optional number of classes to classify images into, only to be specified if include_top is True, and if no weights argument is specified.


In [42]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [43]:
last_layer = model.get_layer('fc2').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)

In [44]:
from keras.models import Model

custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [45]:
for layer in custom_vgg_model.layers[:-1]:
    layer.trainable = False

In [46]:
custom_vgg_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [47]:
hist = custom_vgg_model.fit(X_train, y_train, batch_size=32, epochs=num_epoch, verbose=1, validation_data=(X_test, y_test))

Train on 646 samples, validate on 162 samples
Epoch 1/2
646/646 [==============================] - 237s - loss: 1.7545 - acc: 0.4272 - val_loss: 1.8277 - val_acc: 0.3333
Epoch 2/2
646/646 [==============================] - 236s - loss: 1.0197 - acc: 0.6238 - val_loss: 0.9011 - val_acc: 0.5988


In [48]:
(loss, accuracy) = custom_vgg_model.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))

162/162 [==============================] - 46s    
[INFO] loss=0.9011, accuracy: 59.8765%


###### Transfer Learning - 2

In [49]:
# Training the feature extraction also
model = VGG16(input_tensor=image_input, include_top=True, weights='imagenet')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [50]:
last_layer = model.get_layer('block5_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_vgg_model2 = Model(image_input, out)

In [51]:
custom_vgg_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [52]:
# freeze all the layers except the dense layers
for layer in custom_vgg_model2.layers[:-3]:
    layer.trainable = False

In [53]:
custom_vgg_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [54]:
custom_vgg_model2.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [55]:
hist = custom_vgg_model2.fit(X_train, y_train, batch_size=32, epochs=num_epoch, verbose=1, validation_data=(X_test, y_test))

Train on 646 samples, validate on 162 samples
Epoch 1/2
646/646 [==============================] - 232s - loss: 1.2115 - acc: 0.5248 - val_loss: 0.7870 - val_acc: 0.6605
Epoch 2/2
646/646 [==============================] - 240s - loss: 0.4948 - acc: 0.8142 - val_loss: 0.4437 - val_acc: 0.8086


In [56]:
(loss, accuracy) = custom_vgg_model2.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

162/162 [==============================] - 47s    
[INFO] loss=0.4437, accuracy: 80.8642%
